### with this script you can add attributes from raster or polygons to points 

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

import geopandas
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point

import ee
import geemap
import pandas as pd

In [2]:
# Initialize Earth Engine
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


##### read points

In [3]:
points = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2015_2022_2023_clean.csv', delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2015_2022_2023_prov.csv', delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/GAB_all_ceo_2015_2022_2023_clean.csv', delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/CMR/CMR_all_ceo_2015_2022_2023_clean.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2015_2022_2023_prov.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2015_2022_2023_prov.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/CAR/CAR_all_ceo_2015_2022_2023_pref.csv',delimiter=',', low_memory=False)
#read GEE points
#points = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_cmr')

In [4]:
len(points)

15338

In [5]:
all_columns = points.columns.tolist()

# Display the list of column names
print(all_columns)

['UID', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrov

In [6]:
#drop any columns if needed
#esbae = esbae.drop('index_right', axis=1)

In [7]:
#enter Lat and Lon columns
LATcol = 'lat'
LONcol = 'lon'

In [8]:
gdf = geopandas.GeoDataFrame(
    points, geometry=geopandas.points_from_xy(points[LONcol], points[LATcol]), crs="EPSG:4326")

#### add attributes from local shapefile

In [9]:
poly = "/home/sepal-user/data/admin/RDC_Province_26.shp"
#poly = "/home/sepal-user/data/admin/RCA_adm1_itos.shp"
#poly = "/home/sepal-user/data/admin/cmr_admbnda_adm1_inc_20180104.shp"
#poly = "/home/sepal-user/data/admin/geoBoundaries-GNQ-ADM1.shp"
#poly = "/home/sepal-user/data/admin/geoBoundaries-GAB-ADM1.shp"
# Read file using gpd.read_file()
poly_shp = gpd.read_file(poly)
poly_shp.head()

,SCE_SEM,SCE_GEO,MODIF,ORIGINE,NOM,CODE_INS,SURFACE,CHEF_LIEU,ID_SEPAL,BLABLA,geometry
0,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin / Image satellite,Kinshasa,10,10584.151287,Kinshasa,1,0,"POLYGON ((1775596.089 -434954.674, 1775691.545..."
1,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Kongo Central,20,54493.327517,Matadi,2,0,"POLYGON ((1604421.487 -477541.359, 1605095.95 ..."
2,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin,Mai-Ndombe,302,128788.166437,Inongo,3,0,"POLYGON ((2132294.536 -90899.292, 2133985.101 ..."
3,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin / Image satellite,Kwilu,303,79747.494200,Bandundu,4,0,"POLYGON ((1936677.519 -353809.382, 1936960.249..."
4,INS,UNDP/OCHA,2015-04-28,Numerisation Saint Moulin / Image satellite,Kwango,305,91102.901904,Kenge,5,0,"POLYGON ((1960581.946 -468508.291, 1961890.275..."


In [10]:
#admin_name = 'ADM1_FR'
#admin_name = 'shapeName'
#admin_name = 'admin1Name'
admin_name = 'NOM'
#new_name = 'Prefecture'
new_name = 'Province'

In [11]:
poly_crs = poly_shp.crs
print("Current CRS:", poly_crs)

Current CRS: PROJCS["World_Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_2SP"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","54004"]]


In [12]:
#poly_shp = poly_shp.to_crs(gdf.crs)

In [13]:
# Reproject both GeoDataFrames to the same projected CRS (e.g., EPSG:3395 for World Mercator)
gdf = gdf.to_crs("EPSG:3395")
poly_shp = poly_shp.to_crs("EPSG:3395")

# Step 1: Perform the spatial join for intersecting points
joined_data_intersecting = gpd.sjoin(gdf, poly_shp[[admin_name, 'geometry']], how="left", predicate="within", lsuffix='left', rsuffix='right')

# Drop the 'index_right' column if it exists, to avoid conflicts
if 'index_right' in joined_data_intersecting.columns:
    joined_data_intersecting = joined_data_intersecting.drop(columns='index_right')

# Step 2: Identify points without an intersection (NaN values in the Admin_Name column)
no_intersection_points = joined_data_intersecting[joined_data_intersecting[admin_name].isna()]

# Step 3: Perform the nearest spatial join for points without an intersection
nearest_join = gpd.sjoin_nearest(no_intersection_points.drop(columns=admin_name), poly_shp[[admin_name, 'geometry']], how="left", distance_col="distance_to_polygon")

# Drop the 'index_right' column if it exists in the nearest join result
if 'index_right' in nearest_join.columns:
    nearest_join = nearest_join.drop(columns='index_right')

# Step 4: Combine intersecting and nearest joined data into one GeoDataFrame
# Retain only the specified `admin_name` column from `poly_shp`
joined_data_combined = pd.concat([
    joined_data_intersecting.dropna(subset=[admin_name]),
    nearest_join
])

# Select only the columns from `gdf` plus `admin_name`
columns_to_keep = list(gdf.columns) + [admin_name]
joined_data = joined_data_combined[columns_to_keep]

# Display the resulting GeoDataFrame
joined_data.head()

,UID,ID,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,...,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,Ref_Change_Type_1523,Ref_Change_Year_1523,geometry,NOM
0,0,0,807773,807773.0,807773,12.741453,-5.776823,POINT(12.741453 -5.776823),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Def,Def2016,POINT (1418372.06 -639867.527),Kongo Central
1,1,1,2771794,2771794.0,2771794,25.879949,-3.186465,POINT(25.879949 -3.186465),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Def,Def2020,POINT (2880942.744 -352525.262),Maniema
2,2,2,2883802,2883802.0,2883802,25.220591,-6.717631,POINT(25.220591 -6.717631),0,non,...,NaN,NaN,NaN,NaN,NaN,NaN,NF,NF,POINT (2807543.348 -744527.663),Haut-Lomami
3,3,3,2880499,2880499.0,2880499,23.853181,-10.566491,POINT(23.853181 -10.566491),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (2655323.963 -1175150.882),Lualaba
4,4,4,2877893,2877893.0,2877893,23.572110,-10.311778,POINT(23.57211 -10.311778),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Def,Def2021,POINT (2624035.282 -1146505.826),Lualaba


In [14]:
nearest_join.head()

,UID,ID,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,...,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,Ref_Change_Type_1523,Ref_Change_Year_1523,geometry,NOM,distance_to_polygon
133,133,133,3290107,3290107.0,3290107,28.562642,-9.015678,POINT(28.562642 -9.015678),0,non,...,NaN,NaN,NaN,NaN,NaN,NF,NF,POINT (3179578.763 -1001096.829),Haut-Katanga,9899.934918
180,180,180,3368885,3368885.0,3368885,30.298276,-7.340184,POINT(30.298276 -7.340184),0,non,...,NaN,NaN,NaN,NaN,NaN,NF,NF,POINT (3372788.656 -813894.589),Tanganyka,12815.488044
181,181,181,3367758,3367758.0,3367758,30.184052,-7.058320,POINT(30.184052 -7.05832),0,non,...,NaN,NaN,NaN,NaN,NaN,NF,NF,POINT (3360073.299 -782476.683),Tanganyka,19090.930994
185,185,185,3161083,3161083.0,3161083,30.422556,1.253248,POINT(30.422556 1.253248),0,non,...,NaN,NaN,NaN,NaN,NaN,NF,NF,POINT (3386623.443 138588.189),Ituri,467.980607
192,192,192,3233572,3233572.0,3233572,29.227196,-4.549795,POINT(29.227196 -4.549795),0,non,...,NaN,NaN,NaN,NaN,NaN,NF,NF,POINT (3253556.576 -503626.935),Sud-Kivu,3627.225832


In [15]:
# Perform the spatial join using the predicate parameter
#joined_data = gpd.sjoin(gdf, poly_shp, how="left", predicate="within", lsuffix='left', rsuffix='right')
# 'joined_data' now contains the points with additional information from the polygons
# Print or explore the resulting GeoDataFrame
#joined_data.head()

In [16]:
joined_data[admin_name].value_counts(dropna=False)

NOM
Haut-Katanga      1529
Tanganyka         1380
Lualaba           1334
Haut-Lomami        981
Kwango             857
Haut-Uele          845
Bas-Uele           830
Kwilu              654
Kasai              620
Maniema            585
Mai-Ndombe         542
Tshopo             539
Kasai Central      514
Sankuru            457
Kongo Central      443
Sud-Kivu           440
Sud-Ubangi         374
Mongala            372
Ituri              363
Lomami             332
Tshuapa            321
Nord-Kivu          313
Equateur           305
Nord-Ubangi        289
Kasai Oriental      71
Kinshasa            48
Name: count, dtype: int64

In [17]:
province_pts = pd.pivot_table(joined_data,values='plotid',index=[admin_name],columns=['collection'],aggfunc="count",margins=True,
                             margins_name='Total',dropna=False)
province_pts

collection,coll_1522_1,coll_1522_2,coll_1522_3,coll_2023_1,coll_2023_2,coll_2023_3,coll_2023_4,coll_2023_5,coll_2023_6,Total
NOM,,,,,,,,,,
Bas-Uele,33.0,315.0,362.0,21.0,61.0,3.0,14.0,11.0,10.0,830
Equateur,23.0,107.0,118.0,14.0,32.0,1.0,1.0,5.0,4.0,305
Haut-Katanga,39.0,715.0,636.0,27.0,65.0,NaN,7.0,24.0,16.0,1529
Haut-Lomami,21.0,484.0,339.0,42.0,50.0,NaN,16.0,25.0,4.0,981
Haut-Uele,21.0,351.0,355.0,30.0,53.0,2.0,9.0,16.0,8.0,845
Ituri,18.0,164.0,125.0,12.0,28.0,1.0,4.0,6.0,5.0,363
Kasai,15.0,244.0,249.0,17.0,50.0,NaN,18.0,18.0,9.0,620
Kasai Central,14.0,208.0,210.0,15.0,27.0,NaN,13.0,20.0,7.0,514
Kasai Oriental,1.0,43.0,14.0,3.0,7.0,NaN,1.0,2.0,NaN,71


In [18]:
# Remove rows where the 'Province' column has NaN values
joined_data = joined_data.dropna(subset=[admin_name])

In [19]:
len(joined_data)

15338

In [20]:
poly_shp = poly_shp.to_crs("EPSG:3395")

In [21]:
#calculate area of poly
poly_shp['area_sqm'] = poly_shp.geometry.area

In [22]:
poly_shp['area_ha'] = poly_shp['area_sqm'] / 10000

In [23]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# Display the GeoDataFrame with the new area columns
print(poly_shp[[admin_name, 'area_ha']])

               NOM  area_ha
0         Kinshasa  1058415
1    Kongo Central  5449333
2       Mai-Ndombe 12878817
3            Kwilu  7974749
4           Kwango  9110290
5         Equateur 10207799
6       Sud-Ubangi  5220098
7      Nord-Ubangi  5444914
8          Mongala  5625156
9          Tshuapa 13298111
10          Tshopo 20056004
11        Bas-Uele 14923195
12       Haut-Uele  9166490
13           Ituri  6283610
14       Nord-Kivu  5724051
15        Sud-Kivu  5931496
16         Maniema 12803642
17         Lualaba 12545160
18     Haut-Lomami 11137497
19       Tanganyka 12254047
20    Haut-Katanga 12652867
21  Kasai Oriental  1031690
22         Sankuru 10867105
23          Lomami  5461308
24   Kasai Central  5776514
25           Kasai  9748568


In [24]:
# Rename columns
column_mapping = {
   admin_name: new_name
#    'NAME_2': 'Territoire',
#    'Unnamed: 0':'Index'
}

# Use the rename() method to rename columns
joined_data.rename(columns=column_mapping, inplace=True)

In [25]:
list_columns = joined_data.columns.tolist()

# Display the list of column names
print(list_columns)

['UID', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrov

In [26]:
joined_data = joined_data.to_crs("EPSG:4326")

In [27]:
# if no more data are to be joined, save output
out_file_shp_att =  '/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2015_2022_2023_prov.csv'
joined_data.to_csv(out_file_shp_att,index=False)

#### get attributes from local raster

In [ ]:
LC_map = "/home/sepal-user/data/lc_map/CAFI_LC_2015_19_wgs84.tif"

In [ ]:
LC_src = rasterio.open(LC_map)

In [ ]:
joined_data["CAFI_LC_2015"] = [x for x in LC_src.sample(coord_list)]
joined_data["CAFI_LC_2015"] = joined_data["CAFI_LC_2015"].str[0]
joined_data.head()

In [ ]:
joined_data['CAFI_LC_2015'].value_counts(dropna=False)

In [ ]:
joined_columns = joined_data.columns.tolist()
# Display the list of column names
print(joined_columns)

In [ ]:
joined_data.head()

#### get attributes from GEE asset

In [28]:
#first time
gdf = joined_data

In [69]:
#second time
gdf = merged_df

In [70]:
# Assuming df is your DataFrame
if gdf['UID'].nunique() == len(gdf):
    print("UID has entirely unique values.")
else:
    print("UID does not have entirely unique values.")

if gdf['ID'].nunique() == len(gdf):
    print("ID has entirely unique values.")
else:
    print("ID does not have entirely unique values.")

UID has entirely unique values.
ID does not have entirely unique values.


In [71]:
#path to asset
#asset = ee.Image('users/faocongo/sbae/EQG_FNF_kmeans_strat_5_2015_2022')
#asset = ee.Image('users/faocongo/sbae/EQG_FNF_kmeans_strat_5_2022_2023')
#asset = ee.Image('users/faocongo/sbae/CAF_FNF_kmeans_strat_5_2015_2022')
#asset = ee.Image('users/faocongo/sbae/CAF_FNF_kmeans_strat_5_2022_2023')
#asset = ee.Image('users/faocongo/sbae/GAB_FNF_kmeans_strat_5_2015_2022')
#asset = ee.Image('users/faocongo/sbae/GAB_FNF_kmeans_strat_5_2022_2023')
asset = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2015_2022_v2')
#asset = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2022_2023')
#asset = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2022_2023_v2')
#asset = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2022_2023_v3')
#asset = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2022_2023_v4')
#asset = ee.Image('users/faocongo/sbae/COD_FNF_kmeans_strat_5_2015_2022_v2')
#asset = ee.Image('users/faocongo/sbae/CMR_FNF_kmeans_strat_5_2015_2022')
#asset = ee.Image('users/faocongo/sbae/CMR_FNF_kmeans_strat_5_2022_2023')


In [72]:
#what to rename sampled band to. it is always called first
newName = 'TNT_stratum_1522_v2'
#newName = 'TNT_stratum_2223_v3'

In [73]:
#resolution of asset
scale = 70

In [74]:
#name of band
band = 'strata'

In [75]:
gdf = gdf.reset_index(drop=True)
#with just a few columns
#this can take a little while with many points
dfs = []
for i in range(0, len(points), 5000):
    tmp_df = gdf.loc[i:i+4999]
    tmp_fc = geemap.gdf_to_ee(tmp_df[['UID', 'geometry']])
    tmp_smp = asset.select(band).reduceRegions(**{
        'collection': tmp_fc,
        'scale': scale,
        'reducer': ee.Reducer.first()   
    })

    tmp_res = geemap.ee_to_gdf(tmp_smp)
    dfs.append(tmp_res)
    
sampled_df = pd.concat(dfs)

In [76]:
sampled_df

,geometry,UID,first
0,POINT (12.74145 -5.77682),0,3
1,POINT (25.87995 -3.18646),1,1
2,POINT (25.22059 -6.71763),2,0
3,POINT (23.85318 -10.56649),3,3
4,POINT (23.57211 -10.31178),4,5
...,...,...,...
333,POINT (28.53135 -9.48868),14053,4
334,POINT (21.80045 4.27683),14163,1
335,POINT (23.81708 4.82014),14235,2
336,POINT (14.99661 -4.59142),14307,3


In [77]:
sampled_df = sampled_df.rename(columns={'first':newName})
sampled_df.head()

,geometry,UID,TNT_stratum_1522_v2
0,POINT (12.74145 -5.77682),0,3
1,POINT (25.87995 -3.18646),1,1
2,POINT (25.22059 -6.71763),2,0
3,POINT (23.85318 -10.56649),3,3
4,POINT (23.57211 -10.31178),4,5


In [78]:
# recover columns
# Step 1: Merge the two DataFrames on the 'ID' column (assuming 'ID' is the common key)
merged_df = pd.merge(sampled_df, gdf, on='UID', how='left', suffixes=('', '_dup'))

# Step 2: Remove duplicate columns
# Here, we assume that any duplicate column from df2 will have the '_dup' suffix added by the merge
# We will keep the columns from df1 and drop the duplicate ones from df2.
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]
merged_df.head()

,geometry,UID,TNT_stratum_1522_v2,TNT_stratum_1522,TNT_stratum_2223_v3,TNT_stratum_2223_v2,TNT_stratum_2223,ID,plotid,point_id,...,Ref_Ftype_2022,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,Ref_Change_Type_1523,Ref_Change_Year_1523,Province
0,POINT (12.74145 -5.77682),0,3,4,2,1,2,0,807773,807773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Def,Def2016,Kongo Central
1,POINT (25.87995 -3.18646),1,1,4,3,2,3,1,2771794,2771794,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Def,Def2020,Maniema
2,POINT (25.22059 -6.71763),2,0,1,0,0,1,2,2883802,2883802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NF,NF,Haut-Lomami
3,POINT (23.85318 -10.56649),3,3,1,4,5,4,3,2880499,2880499,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,Lualaba
4,POINT (23.57211 -10.31178),4,5,3,2,1,2,4,2877893,2877893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Def,Def2021,Lualaba


##### repeat above as needed

In [79]:
merged_df_cols = merged_df.columns.tolist()
# Display the list of column names
print(merged_df_cols)

['geometry', 'UID', 'TNT_stratum_1522_v2', 'TNT_stratum_1522', 'TNT_stratum_2223_v3', 'TNT_stratum_2223_v2', 'TNT_stratum_2223', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', '

In [19]:
#drop any columns if needed
columns_to_drop = ['sample_geom']
merged_df = merged_df.drop(columns_to_drop, axis=1)

KeyError: "['sample_geom'] not found in axis"

In [22]:
# Using loc to replace only rows where 'stratum' is 0
merged_df.loc[merged_df['TNT_stratum_1522'] == 0, 'TNT_stratum_1522'] = 1

In [19]:
# Using loc to replace only rows where 'stratum' is 0
merged_df.loc[merged_df['TNT_stratum_2223_v2'] == 0, 'TNT_stratum_2223_v2'] = 1

In [80]:
merged_df['TNT_stratum_1522'] = merged_df['TNT_stratum_1522'].fillna(1)
merged_df['TNT_stratum_2223'] = merged_df['TNT_stratum_2223'].fillna(1)

In [81]:
merged_df['TNT_stratum_1522_v2'] = merged_df['TNT_stratum_1522_v2'].fillna(1)
merged_df['TNT_stratum_2223_v2'] = merged_df['TNT_stratum_2223_v2'].fillna(1)

In [83]:
merged_df['TNT_stratum_2223_v3'] = merged_df['TNT_stratum_2223_v3'].fillna(1)
#merged_df['TNT_stratum_2223_v4'] = merged_df['TNT_stratum_2223_v4'].fillna(1)

In [84]:
print(merged_df.dtypes)

geometry                geometry
UID                        int64
TNT_stratum_1522_v2      float64
TNT_stratum_1522         float64
TNT_stratum_2223_v3        int64
                          ...   
Deg2023                  float64
interpreted_qc           float64
Ref_Change_Type_1523      object
Ref_Change_Year_1523      object
Province                  object
Length: 114, dtype: object


In [85]:
pd.pivot_table(
    merged_df,
    values='UID',
    index=['TNT_stratum_1522'],
    columns=['collection'],
    aggfunc="count",
    margins=True,              # Adds totals
    margins_name='Total',      # Name of the margins/total column
    dropna=False               # Include NaN values in the table
)

collection,coll_1522_1,coll_1522_2,coll_1522_3,coll_2023_1,coll_2023_2,coll_2023_3,coll_2023_4,coll_2023_5,coll_2023_6,Total
TNT_stratum_1522,,,,,,,,,,
1,181,1892,934,125,318,3,28,40,52,3573
2,118,1421,1325,134,276,7,35,57,47,3420
3,107,1594,1680,128,200,3,60,79,44,3895
4,64,1279,1490,85,156,3,53,80,29,3239
5,28,492,542,26,48,NaN,23,42,10,1211
Total,498,6678,5971,498,998,16,199,298,182,15338


In [86]:
pd.pivot_table(
    merged_df,
    values='UID',
    index=['TNT_stratum_2223'],
    columns=['collection'],
    aggfunc="count",
    margins=True,              # Adds totals
    margins_name='Total',      # Name of the margins/total column
    dropna=False               # Include NaN values in the table
)

collection,coll_1522_1,coll_1522_2,coll_1522_3,coll_2023_1,coll_2023_2,coll_2023_3,coll_2023_4,coll_2023_5,coll_2023_6,Total
TNT_stratum_2223,,,,,,,,,,
1,252,2225,1538,155,441,10,11,14,78,4724
2,98,1758,1483,124,223,3,31,50,46,3816
3,78,1440,1504,119,156,2,33,65,32,3429
4,55,1021,1141,71,122,1,71,89,23,2594
5,15,234,305,29,56,NaN,53,80,3,775
Total,498,6678,5971,498,998,16,199,298,182,15338


In [87]:
# Remove rows where the stratum has NaN values - outside the country
#merged_df = merged_df.dropna(subset=['TNT_stratum_2223'])

In [88]:
merged_df['sampling'].value_counts(dropna=False)

sampling
ceo_1522    13147
ceo_2023     2191
Name: count, dtype: int64

In [89]:
# Check if the lengths of merged_df and ceo are the same
if len(merged_df) == len(points):
    print("The lengths of merged_df and ceo are the same.")
else:
    print("The lengths of merged_df and ceo are different.")

# Optionally, you can print the lengths for clarity
print("Length of merged_df:", len(merged_df))
print("Length of ceo:", len(points))

The lengths of merged_df and ceo are the same.
Length of merged_df: 15338
Length of ceo: 15338


#### export

In [90]:
out_file =  '/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2015_2022_2023_TNTstrat.csv'
#sampled_df.to_csv(out_file,index=False)
merged_df.to_csv(out_file,index=False)